In [ ]:
import numpy as np
from OpenEphys import loadContinuous
import os
import glob
from scipy import signal as ss
from scipy import stats as st
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#Select file for analysis
files_to_be_converted = ['Pre 1','Pre 2','Pre 3','Cit 1','Cit 2','Cit 3','Way 1', 'Way 2','Way 3']
source_folder = r'/Users/sharplab/tran/EEG data'

path = "/".join([source_folder,files_to_be_converted[0]])
os.chdir(path)
file = glob.glob("*.continuous")
print(file[0])
eegdata = loadContinuous(file[0], dtype=float)
array = eegdata['data']

#Resampling the data
duration = np.shape(array)[0]/30000
newarrayshape = np.real(int(round(250*duration)))
newarray=ss.resample(array, newarrayshape)

#Functions for filters
def butter_lowpass(low_cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_low_cutoff = low_cutoff / nyq
    b_low, a_low = butter(order, normal_low_cutoff, btype='lowpass', analog=False)
    return b_low, a_low

def butter_lowpass_filter(data, low_cutoff, fs, order=5):
    b_low, a_low = butter_lowpass(low_cutoff, fs, order=order)
    y_low = lfilter(b_low, a_low, data)
    return y_low

# Filter requirements.
order = 5
fs = 30000.0       # sample rate, Hz
low_cutoff = 100 # desired cutoff frequency of the filter, Hz
high_cutoff = 0.1


# Filter the data.
print('Filtering data')
filtered_array = butter_lowpass_filter(newarray, low_cutoff, fs, order)


#Spectrogram
sampling_rate = 250
bin_size = sampling_rate * 4

binned_frequencies, binned_time, binned_Sxx = ss.spectrogram(filtered_array,
                                                             sampling_rate, ss.get_window('hamming', bin_size), noverlap=bin_size/2, nperseg=bin_size)

#Dataframe of the spectrogram results
df = pd.DataFrame(data=binned_Sxx,
                     columns=binned_time,
                     index=binned_frequencies).transpose()
df = df.reset_index()
df = df.rename(columns={'index': 'time'})
df['time'] = pd.to_timedelta(df['time'], unit='s')
df.set_index('time', inplace=True)
df.head()

#Plot the end result
df_lowf = df.iloc[:, :]
mean_freqs = df_lowf.apply(np.mean, axis=0)
plt.plot(mean_freqs.index, np.log(mean_freqs))
plt.show()

100_CH43.continuous
Loading continuous data...
